# Human-in-the-Loop Guessing Game

## Overview

This notebook demonstrates **human-in-the-loop (HITL)** workflows - a pattern where an AI agent alternates turns with a human to collaboratively solve a problem. This interactive guessing game showcases:

1. **RequestInfoExecutor**: Pause workflow for human input
2. **Request/Response Correlation**: Match human replies to specific requests
3. **Structured Agent Output**: Enforce JSON schema with `response_format`
4. **Turn-Based Coordination**: Orchestrate agent ↔ human cycles
5. **Event-Driven Streaming**: Real-time interaction via workflow events

### Game Flow:

```
Start Game
    ↓
┌─────────────────────────────┐
│  Agent Makes Guess (1-10)   │
└─────────────────────────────┘
    ↓
TurnManager requests human feedback
    ↓
┌─────────────────────────────┐
│  Human Responds:            │
│  • "higher" - guess low     │
│  • "lower" - guess high     │
│  • "correct" - game over ✓  │
│  • "exit" - quit game       │
└─────────────────────────────┘
    ↓
Agent adjusts guess (repeat until correct)
```

### Key Architecture Components:

1. **TurnManager (Custom Executor)**
   - Coordinates game flow
   - Handles agent responses
   - Processes human feedback
   - Determines game completion

2. **AgentExecutor**
   - Wraps Azure OpenAI ChatAgent
   - Generates guesses with structured output
   - Adjusts based on feedback

3. **RequestInfoExecutor**
   - Pauses workflow for human input
   - Emits `RequestInfoEvent` with typed request
   - Resumes when application provides `RequestResponse`
   - Maintains request/response correlation

### Workflow Graph:

```
TurnManager ─(start)──> AgentExecutor
     ↑                        |
     |                   (response)
     |                        ↓
     └────────────────── TurnManager
                              |
                      (request feedback)
                              ↓
                      RequestInfoExecutor
                              |
                       (human response)
                              ↓
                         TurnManager
```

## Prerequisites

- Azure OpenAI configured with environment variables:
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_API_VERSION`
  - `AZURE_OPENAI_DEPLOYMENT_NAME`
- Azure CLI authentication: `az login`
- Agent Framework installed: `pip install agent-framework`
- Interactive environment for human input (Jupyter notebook supports `input()`)

## Setup and Imports

In [ ]:
import asyncio
from dataclasses import dataclass

import os
from dotenv import load_dotenv
from agent_framework import (
    AgentExecutor,  # Executor that runs the agent
    AgentExecutorRequest,  # Message bundle sent to an AgentExecutor
    AgentExecutorResponse,  # Result returned by an AgentExecutor
    ChatMessage,  # Chat message structure
    Executor,  # Base class for workflow executors
    RequestInfoEvent,  # Event emitted when human input is requested
    RequestInfoExecutor,  # Special executor that collects human input out of band
    RequestInfoMessage,  # Base class for request payloads sent to RequestInfoExecutor
    RequestResponse,  # Correlates a human response with the original request
    Role,  # Enum of chat roles (user, assistant, system)
    WorkflowBuilder,  # Fluent builder for assembling the graph
    WorkflowContext,  # Per run context and event bus
    WorkflowOutputEvent,  # Event emitted when workflow yields output
    WorkflowRunState,  # Enum of workflow run states
    WorkflowStatusEvent,  # Event emitted on run state changes
    handler,  # Decorator to expose an Executor method as a step
)
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
from pydantic import BaseModel

# Load environment variables from .env file
load_dotenv('../../.env')


## Define Request and Response Models

### HumanFeedbackRequest

Subclasses `RequestInfoMessage` to define the schema for human input requests.

**Why subclass RequestInfoMessage?**
- Strong typing for request payloads
- Forward-compatible validation
- Clear correlation semantics
- Attach contextual fields (e.g., agent's previous guess)
- Rich UI rendering without extra state fetching

**Fields:**
- `prompt`: Instructions for the human
- `guess`: Agent's last guess (provides context)

### GuessOutput

Pydantic model for structured agent output.

**Benefits of `response_format`:**
- Enforces JSON schema compliance
- Eliminates regex parsing
- Reliable, type-safe output
- Automatic validation

In [ ]:
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """Request payload sent to RequestInfoExecutor for human feedback.
    
    Including the agent's last guess allows the UI to display context
    and helps the turn manager avoid extra state reads.
    """
    prompt: str = ""
    guess: int | None = None


class GuessOutput(BaseModel):
    """Structured output from the agent. Enforced via response_format for reliable parsing."""
    guess: int

## Create TurnManager - The Game Coordinator

The `TurnManager` custom executor orchestrates the entire game flow.

### Handler Methods:

#### 1. `start()`
- **Trigger**: Workflow initialization with "start" message
- **Action**: Send initial request to agent for first guess
- **Output**: `AgentExecutorRequest` with user message

#### 2. `on_agent_response()`
- **Trigger**: Receives `AgentExecutorResponse` from agent
- **Action**: 
  - Parse structured JSON output (`GuessOutput`)
  - Create human feedback request
  - Send `HumanFeedbackRequest` to RequestInfoExecutor
- **Output**: Request for human to provide guidance

#### 3. `on_human_feedback()`
- **Trigger**: Receives `RequestResponse[HumanFeedbackRequest, str]` from human
- **Action**:
  - Process human reply ("higher", "lower", "correct")
  - If "correct": Yield output and complete workflow
  - If guidance: Send feedback to agent for next guess
- **Output**: Either workflow output (game over) or next agent request

### Design Pattern: Request/Response Correlation

The `RequestResponse` object contains:
- **`data`**: Human's string reply
- **`original_request`**: The correlated `HumanFeedbackRequest`

This avoids needing shared state - all context is in the message!

In [ ]:
class TurnManager(Executor):
    """Coordinates turns between the agent and the human.

    Responsibilities:
    - Kick off the first agent turn.
    - After each agent reply, request human feedback with a HumanFeedbackRequest.
    - After each human reply, either finish the game or prompt the agent again with feedback.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "turn_manager")

    @handler
    async def start(self, _: str, ctx: WorkflowContext[AgentExecutorRequest]) -> None:
        """Start the game by asking the agent for an initial guess.

        Contract:
        - Input is a simple starter token (ignored here).
        - Output is an AgentExecutorRequest that triggers the agent to produce a guess.
        """
        user = ChatMessage(Role.USER, text="Start by making your first guess.")
        await ctx.send_message(AgentExecutorRequest(messages=[user], should_respond=True))

    @handler
    async def on_agent_response(
        self,
        result: AgentExecutorResponse,
        ctx: WorkflowContext[HumanFeedbackRequest],
    ) -> None:
        """Handle the agent's guess and request human guidance.

        Steps:
        1) Parse the agent's JSON into GuessOutput for robustness.
        2) Send a HumanFeedbackRequest to the RequestInfoExecutor with a clear instruction:
           - higher means the human's secret number is higher than the agent's guess.
           - lower means the human's secret number is lower than the agent's guess.
           - correct confirms the guess is exactly right.
           - exit quits the demo.
        """
        # Parse structured model output (defensive default if the agent did not reply).
        text = result.agent_run_response.text or ""
        last_guess = GuessOutput.model_validate_json(text).guess if text else None

        # Craft a precise human prompt that defines higher and lower relative to the agent's guess.
        prompt = (
            f"The agent guessed: {last_guess if last_guess is not None else text}. "
            "Type one of: higher (your number is higher than this guess), "
            "lower (your number is lower than this guess), correct, or exit."
        )
        await ctx.send_message(HumanFeedbackRequest(prompt=prompt, guess=last_guess))

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest, str],
    ) -> None:
        """Continue the game or finish based on human feedback.

        The RequestResponse contains both the human's string reply and the correlated HumanFeedbackRequest,
        which carries the prior guess for convenience.
        """
        reply = (feedback.data or "").strip().lower()
        # Prefer the correlated request's guess to avoid extra shared state reads.
        last_guess = getattr(feedback.original_request, "guess", None)

        if reply == "correct":
            await ctx.yield_output(f"Guessed correctly: {last_guess}")
            return

        # Provide feedback to the agent to try again.
        # We keep the agent's output strictly JSON to ensure stable parsing on the next turn.
        user_msg = ChatMessage(
            Role.USER,
            text=(f'Feedback: {reply}. Return ONLY a JSON object matching the schema {{"guess": <int 1..10>}}.'),
        )
        await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))

## Understanding RequestInfoExecutor

### What RequestInfoExecutor Does:

`RequestInfoExecutor` is a **workflow-native bridge** that:

1. **Pauses the workflow graph** at a request for information
2. **Emits `RequestInfoEvent`** with a typed payload (e.g., `HumanFeedbackRequest`)
3. **Resumes the workflow** only after application supplies matching `RequestResponse`
4. **Maintains correlation** via `request_id` key

### What RequestInfoExecutor Does NOT Do:

- **Does NOT gather input itself** - Your application is responsible
- **Does NOT have a UI** - You build the interface
- **Does NOT store responses** - You provide them via `send_responses_streaming()`

### Benefits:

- **Standardizes pause-and-resume** human gating
- **Carries typed request payloads** (via RequestInfoMessage subclasses)
- **Preserves correlation** (request_id → response mapping)
- **Integrates seamlessly** with workflow graph

### Request/Response Flow:

```python
# 1. Workflow emits request
await ctx.send_message(HumanFeedbackRequest(prompt="...", guess=5))

# 2. RequestInfoExecutor emits RequestInfoEvent
# event.request_id = "req_abc123"
# event.data = HumanFeedbackRequest(prompt="...", guess=5)

# 3. Application collects human input
human_reply = input("Enter higher/lower/correct: ")

# 4. Application provides response
responses = {"req_abc123": human_reply}
await workflow.send_responses_streaming(responses)

# 5. Workflow resumes with RequestResponse
# feedback.data = "higher"
# feedback.original_request = HumanFeedbackRequest(prompt="...", guess=5)
```

## Build the Workflow Graph

### Components:

1. **TurnManager**: Start executor and coordinator
2. **AgentExecutor**: Wraps Azure OpenAI ChatAgent
3. **RequestInfoExecutor**: Human input gateway

### Edges (Message Flow):

```
TurnManager → AgentExecutor
  (sends AgentExecutorRequest to get guess)

AgentExecutor → TurnManager
  (returns AgentExecutorResponse with guess)

TurnManager → RequestInfoExecutor
  (sends HumanFeedbackRequest for guidance)

RequestInfoExecutor → TurnManager
  (returns RequestResponse with human reply)
```

### Agent Configuration:

- **`response_format=GuessOutput`**: Enforces JSON schema
- **Instructions**: Clear guessing rules and format requirements
- **AzureOpenAIChatClient**: Uses Azure CLI credentials

In [ ]:
# Create the chat agent with structured output
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
chat_client = AzureOpenAIChatClient(
    deployment_name=deployment_name,
    endpoint=endpoint,
    credential=AzureCliCredential()
)
agent = chat_client.create_agent(
    instructions=(
        "You guess a number between 1 and 10. "
        "If the user says 'higher' or 'lower', adjust your next guess. "
        'You MUST return ONLY a JSON object exactly matching this schema: {"guess": <integer 1..10>}. '
        "No explanations or additional text."
    ),
    response_format=GuessOutput,
)

# Create workflow executors
turn_manager = TurnManager(id="turn_manager")
agent_exec = AgentExecutor(agent=agent, id="agent")
request_info_executor = RequestInfoExecutor(id="request_info")

# Build the workflow graph
workflow = (
    WorkflowBuilder()
    .set_start_executor(turn_manager)
    .add_edge(turn_manager, agent_exec)  # Ask agent to make/adjust a guess
    .add_edge(agent_exec, turn_manager)  # Agent's response comes back to coordinator
    .add_edge(turn_manager, request_info_executor)  # Ask human for guidance
    .add_edge(request_info_executor, turn_manager)  # Feed human guidance back to coordinator
    .build()
)

print("✓ Workflow built successfully")
print("\nWorkflow Graph:")
print("  TurnManager (start) → AgentExecutor")
print("  AgentExecutor → TurnManager")
print("  TurnManager → RequestInfoExecutor")
print("  RequestInfoExecutor → TurnManager")

## Run the Interactive Game Loop

### Execution Pattern:

The game uses a **streaming request/response loop**:

#### First Iteration:
```python
stream = workflow.run_stream("start")
```

#### Subsequent Iterations:
```python
stream = workflow.send_responses_streaming(pending_responses)
```

### Event Processing:

Each iteration collects events:
- **`RequestInfoEvent`**: Human input needed
- **`WorkflowOutputEvent`**: Game result (completion)
- **`WorkflowStatusEvent`**: State transitions

### Workflow States:

1. **`IN_PROGRESS_PENDING_REQUESTS`**: Requests are being emitted
2. **`IDLE_WITH_PENDING_REQUESTS`**: Workflow paused, awaiting human input
3. **Completion**: `WorkflowOutputEvent` emitted

### Input Collection:

For each `RequestInfoEvent`:
1. Display prompt to human
2. Collect response via `input()`
3. Map `request_id` → human response
4. Pass to `send_responses_streaming()` in next iteration

### Game Instructions:

When prompted, enter one of:
- **`higher`**: Your secret number is higher than the agent's guess
- **`lower`**: Your secret number is lower than the agent's guess
- **`correct`**: Agent guessed your number correctly (game ends)
- **`exit`**: Quit the game early

In [ ]:
async def play_guessing_game():
    """Run the interactive guessing game loop."""
    
    # Game state
    pending_responses: dict[str, str] | None = None
    completed = False
    workflow_output: str | None = None

    # User guidance
    print("=" * 70)
    print("INTERACTIVE GUESSING GAME")
    print("=" * 70)
    print("Think of a number between 1 and 10.")
    print("The AI agent will try to guess it.")
    print("\nWhen prompted, type one of:")
    print("  • higher  - Your number is HIGHER than the agent's guess")
    print("  • lower   - Your number is LOWER than the agent's guess")
    print("  • correct - Agent guessed correctly (game ends)")
    print("  • exit    - Quit the game")
    print("=" * 70 + "\n")

    while not completed:
        # First iteration uses run_stream("start").
        # Subsequent iterations use send_responses_streaming with pending_responses.
        stream = (
            workflow.send_responses_streaming(pending_responses) 
            if pending_responses 
            else workflow.run_stream("start")
        )
        
        # Collect events for this turn
        events = [event async for event in stream]
        pending_responses = None

        # Process events
        requests: list[tuple[str, str]] = []  # (request_id, prompt)
        for event in events:
            if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
                # RequestInfoEvent for our HumanFeedbackRequest
                requests.append((event.request_id, event.data.prompt))
            elif isinstance(event, WorkflowOutputEvent):
                # Capture workflow output as they're yielded
                workflow_output = str(event.data)
                completed = True

        # Display workflow state transitions (for debugging/understanding)
        pending_status = any(
            isinstance(e, WorkflowStatusEvent) and e.state == WorkflowRunState.IN_PROGRESS_PENDING_REQUESTS
            for e in events
        )
        idle_with_requests = any(
            isinstance(e, WorkflowStatusEvent) and e.state == WorkflowRunState.IDLE_WITH_PENDING_REQUESTS
            for e in events
        )
        if pending_status:
            print("[State: IN_PROGRESS_PENDING_REQUESTS - requests outstanding]")
        if idle_with_requests:
            print("[State: IDLE_WITH_PENDING_REQUESTS - awaiting human input]\n")

        # If we have any human requests, prompt the user and prepare responses
        if requests and not completed:
            responses: dict[str, str] = {}
            for req_id, prompt in requests:
                print(f"🤖 {prompt}")
                answer = input("👤 Enter higher/lower/correct/exit: ").strip().lower()
                
                if answer == "exit":
                    print("\n👋 Exiting game...")
                    return
                
                responses[req_id] = answer
                print()  # Blank line for readability
            
            pending_responses = responses

    # Show final result
    print("=" * 70)
    print("🎉 GAME OVER")
    print("=" * 70)
    print(f"Result: {workflow_output}")
    print("=" * 70)


# Run the game
await play_guessing_game()

## Expected Output Example

```
======================================================================
INTERACTIVE GUESSING GAME
======================================================================
Think of a number between 1 and 10.
The AI agent will try to guess it.

When prompted, type one of:
  • higher  - Your number is HIGHER than the agent's guess
  • lower   - Your number is LOWER than the agent's guess
  • correct - Agent guessed correctly (game ends)
  • exit    - Quit the game
======================================================================

[State: IN_PROGRESS_PENDING_REQUESTS - requests outstanding]
[State: IDLE_WITH_PENDING_REQUESTS - awaiting human input]

🤖 The agent guessed: 5. Type one of: higher (your number is higher than this guess), lower (your number is lower than this guess), correct, or exit.
👤 Enter higher/lower/correct/exit: higher

[State: IN_PROGRESS_PENDING_REQUESTS - requests outstanding]
[State: IDLE_WITH_PENDING_REQUESTS - awaiting human input]

🤖 The agent guessed: 8. Type one of: higher (your number is higher than this guess), lower (your number is lower than this guess), correct, or exit.
👤 Enter higher/lower/correct/exit: higher

[State: IN_PROGRESS_PENDING_REQUESTS - requests outstanding]
[State: IDLE_WITH_PENDING_REQUESTS - awaiting human input]

🤖 The agent guessed: 10. Type one of: higher (your number is higher than this guess), lower (your number is lower than this guess), correct, or exit.
👤 Enter higher/lower/correct/exit: lower

[State: IN_PROGRESS_PENDING_REQUESTS - requests outstanding]
[State: IDLE_WITH_PENDING_REQUESTS - awaiting human input]

🤖 The agent guessed: 9. Type one of: higher (your number is higher than this guess), lower (your number is lower than this guess), correct, or exit.
👤 Enter higher/lower/correct/exit: correct

======================================================================
🎉 GAME OVER
======================================================================
Result: Guessed correctly: 9
======================================================================
```

## Key Takeaways

### 1. Human-in-the-Loop Pattern

**Core Components:**

```python
# 1. Define request schema
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    prompt: str
    guess: int | None = None

# 2. Send request in handler
await ctx.send_message(HumanFeedbackRequest(prompt="...", guess=5))

# 3. Add RequestInfoExecutor to workflow
request_info = RequestInfoExecutor(id="request_info")
builder.add_edge(turn_manager, request_info)
builder.add_edge(request_info, turn_manager)

# 4. Process RequestInfoEvent
if isinstance(event, RequestInfoEvent):
    request_id = event.request_id
    prompt = event.data.prompt
    
# 5. Collect human input
human_reply = input(prompt)

# 6. Resume workflow
responses = {request_id: human_reply}
await workflow.send_responses_streaming(responses)
```

### 2. RequestInfoExecutor Capabilities

| Feature | Benefit |
|---------|--------|
| **Typed Requests** | Strong typing via RequestInfoMessage subclasses |
| **Correlation** | Automatic request_id → response mapping |
| **Context Preservation** | RequestResponse includes original request |
| **Workflow Integration** | Standard executor, works with edges |
| **Pause/Resume** | Clean workflow suspension mechanism |

### 3. Request/Response Correlation

**Why correlation matters:**
- No shared state needed
- Context travels with messages
- Multiple concurrent requests supported
- Type-safe request/response pairing

**Example:**
```python
@handler
async def on_human_feedback(
    self,
    feedback: RequestResponse[HumanFeedbackRequest, str],
    ctx: WorkflowContext[...]
) -> None:
    # Access human's reply
    reply = feedback.data  # "higher", "lower", "correct"
    
    # Access original request context
    last_guess = feedback.original_request.guess
    prompt = feedback.original_request.prompt
```

### 4. Structured Output with response_format

**Without `response_format` (fragile):**
```python
# Agent might return: "I guess 5" or "My guess is: 5" or "5"
text = agent_response.text
guess = int(re.search(r'\d+', text).group())  # Brittle regex parsing
```

**With `response_format` (reliable):**
```python
class GuessOutput(BaseModel):
    guess: int

agent = chat_client.create_agent(
    instructions="...",
    response_format=GuessOutput  # Enforce schema
)

# Agent returns: {"guess": 5}
output = GuessOutput.model_validate_json(agent_response.text)
guess = output.guess  # Type-safe, validated
```

### 5. Workflow State Machine

**State Transitions:**

```
START
  ↓
IN_PROGRESS
  ↓
IN_PROGRESS_PENDING_REQUESTS (request emitted)
  ↓
IDLE_WITH_PENDING_REQUESTS (awaiting human)
  ↓ (human provides response)
IN_PROGRESS (resume)
  ↓
COMPLETED (output yielded)
```

**Monitoring states:**
```python
for event in events:
    if isinstance(event, WorkflowStatusEvent):
        if event.state == WorkflowRunState.IDLE_WITH_PENDING_REQUESTS:
            print("Waiting for human input...")
        elif event.state == WorkflowRunState.IN_PROGRESS_PENDING_REQUESTS:
            print("Request being processed...")
```

### 6. TurnManager Design Pattern

**Responsibilities:**
- **Initialize**: Start the interaction
- **Coordinate**: Route messages between agent and human
- **Process**: Handle responses from both parties
- **Decide**: Determine when to continue vs. complete

**Multi-handler pattern:**
```python
class TurnManager(Executor):
    @handler  # Handles str input (start)
    async def start(self, _: str, ctx: WorkflowContext[...]) -> None:
        ...
    
    @handler  # Handles AgentExecutorResponse
    async def on_agent_response(self, result: AgentExecutorResponse, ctx: WorkflowContext[...]) -> None:
        ...
    
    @handler  # Handles RequestResponse[HumanFeedbackRequest, str]
    async def on_human_feedback(self, feedback: RequestResponse[...], ctx: WorkflowContext[...]) -> None:
        ...
```

### 7. Production HITL Implementation

#### Web API Pattern
```python
from fastapi import FastAPI, BackgroundTasks

app = FastAPI()

# Store pending requests
pending_requests: dict[str, dict] = {}

@app.post("/workflow/start")
async def start_workflow(task: str):
    workflow_id = generate_workflow_id()
    # Start workflow in background
    asyncio.create_task(run_workflow(workflow_id, task))
    return {"workflow_id": workflow_id}

@app.get("/workflow/{workflow_id}/pending")
async def get_pending_requests(workflow_id: str):
    return pending_requests.get(workflow_id, [])

@app.post("/workflow/{workflow_id}/respond")
async def submit_response(workflow_id: str, request_id: str, response: str):
    await provide_response(workflow_id, {request_id: response})
    return {"status": "submitted"}
```

#### Notification Pattern
```python
async def notify_user_on_request(event: RequestInfoEvent):
    if isinstance(event.data, HumanFeedbackRequest):
        # Send email/Slack/SMS
        await send_notification(
            user="user@company.com",
            subject="Input Required",
            body=event.data.prompt
        )
```

### 8. Use Cases for Human-in-the-Loop

#### Approval Workflows
- Content review before publishing
- Financial transaction approval
- Data deletion confirmation
- Model deployment gates

#### Guidance and Refinement
- Creative content iteration (this example)
- Research direction steering
- Report outline approval
- Parameter tuning

#### Quality Assurance
- Data validation
- Output verification
- Fact checking
- Compliance review

#### Domain Expertise
- Medical diagnosis confirmation
- Legal document review
- Technical specification approval
- Safety-critical decisions

### 9. Advanced Patterns

#### Multi-User Approval
```python
@dataclass
class ApprovalRequest(RequestInfoMessage):
    content: str
    required_approvers: list[str]
    approvals_needed: int = 2

@handler
async def on_approval_response(
    self,
    response: RequestResponse[ApprovalRequest, dict],
    ctx: WorkflowContext[...]
) -> None:
    approvals = response.data["approvals"]
    if len(approvals) >= response.original_request.approvals_needed:
        await ctx.yield_output("Approved")
    else:
        # Request more approvals
        ...
```

#### Conditional Human Intervention
```python
@handler
async def on_agent_response(
    self,
    result: AgentExecutorResponse,
    ctx: WorkflowContext[...]
) -> None:
    confidence = result.metadata.get("confidence", 1.0)
    
    if confidence < 0.7:
        # Low confidence - request human review
        await ctx.send_message(ReviewRequest(content=result.text))
    else:
        # High confidence - proceed automatically
        await ctx.yield_output(result.text)
```

#### Timeout Handling
```python
import asyncio

async def run_with_timeout():
    try:
        result = await asyncio.wait_for(
            collect_human_response(),
            timeout=300  # 5 minutes
        )
    except asyncio.TimeoutError:
        # Use default or escalate
        result = "default_response"
```

### 10. Comparison with Other Patterns

| Pattern | Human Involvement | Use Case |
|---------|------------------|----------|
| **HITL (This Example)** | Interactive turn-taking | Iterative refinement, games |
| **Plan Review** | Single approval point | Validate approach before execution |
| **Checkpointing** | Resume after pause | Long-running workflows |
| **Tool Approval** | Per-operation gates | Security, compliance |
| **Feedback Loop** | Post-execution review | Quality assurance |

### 11. Testing HITL Workflows

#### Automated Testing
```python
import pytest

@pytest.mark.asyncio
async def test_guessing_game():
    # Mock human responses
    mock_responses = iter(["higher", "lower", "correct"])
    
    async def mock_input(prompt: str) -> str:
        return next(mock_responses)
    
    # Run workflow with mocked input
    result = await run_workflow_with_mock(mock_input)
    assert "Guessed correctly" in result
```

#### Integration Testing
```python
async def test_request_response_correlation():
    events = []
    async for event in workflow.run_stream("start"):
        events.append(event)
        if isinstance(event, RequestInfoEvent):
            break
    
    # Verify request structure
    request_event = events[-1]
    assert isinstance(request_event.data, HumanFeedbackRequest)
    assert request_event.data.guess is not None
    
    # Provide response
    responses = {request_event.request_id: "higher"}
    async for event in workflow.send_responses_streaming(responses):
        # Verify response processing
        pass
```

### 12. Best Practices Summary

#### Request Design
✅ **Do:**
- Subclass `RequestInfoMessage` for type safety
- Include context in request (e.g., `guess` field)
- Clear, actionable prompts
- Validate response format

❌ **Don't:**
- Use generic string requests without structure
- Rely on external state for context
- Ambiguous instructions
- Forget response validation

#### Workflow Design
✅ **Do:**
- Use `response_format` for structured agent output
- Add RequestInfoExecutor edges explicitly
- Handle all event types (RequestInfoEvent, WorkflowOutputEvent, WorkflowStatusEvent)
- Provide clear completion conditions

❌ **Don't:**
- Parse agent output with regex
- Forget to wire RequestInfoExecutor in graph
- Ignore workflow state transitions
- Create infinite loops without exit conditions

#### Production
✅ **Do:**
- Implement timeout handling
- Store requests persistently
- Send notifications for pending requests
- Audit all human responses
- Support multiple concurrent requests

❌ **Don't:**
- Block indefinitely waiting for human
- Lose requests on server restart
- Assume humans will respond immediately
- Skip logging/auditing
- Assume sequential request processing